# Brunel Visualization Cookbook にあるサンプルの表示

以下のサイトにBrunelで作成出来るグラフ等のサンプルコードがあるが、そこには実際の画像が表示されていない。

https://github.com/Brunel-Visualization/Brunel/wiki/Brunel-Visualization-Cookbook

以降では、それらのサンプルコードを使用して実際に画像を表示させてみる。

## 1. 使用するデータの読込み

以下のサイトからBrunelのCookbookで使用されているサンプルデータをDownloadし使用。サイトにあるデータの下部には注釈がついているためDownloadした後に、余分な注釈を削除してからCSVファイルとして保存する。（文字コードはUTF-8）修正したCSVファイルは予めObject Storage上にUnloadしておく。

- http://brunel.mybluemix.net/sample_data/US%20States.csv
- http://brunel.mybluemix.net/sample_data/AirlineDelays.csv
- http://brunel.mybluemix.net/sample_data/Unemployment.csv
- http://brunel.mybluemix.net/sample_data/whiskey.csv
- http://brunel.mybluemix.net/sample_data/BGG%20Top%202000%20Games.csv
- http://brunel.mybluemix.net/sample_data/cpi.csv
- http://brunel.mybluemix.net/sample_data/sierpinski.csv
- http://brunel.mybluemix.net/sample_data/minard-troops.csv

- http://brunel.mybluemix.net/sample_data/minard-cities.csv

Object StorageにデータをUploadしたら、右上の「１００１」（Data）のアイコンをクリックしファイルを表示させ、
適当なファイルの「Insert to code」で「Insert Credentials」を選択し以下のセルに credential を挿入する。
credentials1, credentials2, ... など番号が異なる場合があるが、以降のセルでは credentials2 という名前で参照しているので、名前は credentials2 に変更しておく。

In [1]:
// The code was removed by DSX for sharing.

上のセルと同じく適当なファイルの「Insert to code」で「Insert SparkSession DataFrame」を選択し以下のセルにObject Storageからデータを読み込むコードを挿入する。
dfData1, dfData2, ... など番号が異なる場合があるが、以降のセルではここで読み込むデータフレームは使用しないので特に修正は不要。（実際に必要なのはCloudObjectStorage,
SparkSession を作成するところまでのコード）

In [2]:
// The code was removed by DSX for sharing.

Waiting for a Spark session to start...

[Stage 2:=============================>                             (1 + 1) / 2]+----+----------+----------+--------+----------+--------+------+--------+---------+-----+-------+------+------+----+--------------+------+-------+-------------------+---------+----------+
|Abbr|     State|      Date|  Region|Population|Under 18|  Area|Latitude|Longitude|Water|Density|Summer|Winter|Rain|Violent Crimes|Income|Dem/Rep|Presidential Choice|Boys Name|Girls Name|
+----+----------+----------+--------+----------+--------+------+--------+---------+-----+-------+------+------+----+--------------+------+-------+-------------------+---------+----------+
|  AL|   Alabama|1819-12-14|   South|   4849377|    22.8| 52420|   32.74|   -86.84|  3.5|   92.5|  78.6|  46.5|58.3|           425| 41415|  -22.2|             Romney|  William|       Eva|
|  AK|    Alaska|  1959/1/3| Pacific|    736732|    25.3|665384|   63.35|  -152.84| 16.6|    1.1|  52.3|   2.6|22.5|           688| 69825|  -14.0|             Romney|  

In [72]:
dfData2.filter($"State" === "Hawaii").show()

+----+------+---------+-------+----------+--------+-----+--------+---------+-----+-------+------+------+----+--------------+------+-------+-------------------+---------+----------+
|Abbr| State|     Date| Region|Population|Under 18| Area|Latitude|Longitude|Water|Density|Summer|Winter|Rain|Violent Crimes|Income|Dem/Rep|Presidential Choice|Boys Name|Girls Name|
+----+------+---------+-------+----------+--------+-----+--------+---------+-----+-------+------+------+----+--------------+------+-------+-------------------+---------+----------+
|  HI|Hawaii|1959/8/21|Pacific|   1419561|    21.7|10932|   19.81|  -155.51| 70.2|  129.9|  72.2|  67.4|63.7|           281| 62814|   42.7|              Obama|     Noah|      Emma|
+----+------+---------+-------+----------+--------+-----+--------+---------+-----+-------+------+------+----+--------------+------+-------+-------------------+---------+----------+



以降で実際に使用するデータフレームを読込み

In [50]:
import org.apache.spark.sql.DataFrame

def load_from_COS(csv_filename: String) : DataFrame = {
    spark.
        read.format("org.apache.spark.sql.execution.datasources.csv.CSVFileFormat").
        option("header", "true").
        option("inferSchema", "true").
        load(cos.url(credentials2("BUCKET"), csv_filename))
}

val df_US_States          = load_from_COS("US_States.csv")
val df_AirlineDelays      = load_from_COS("AirlineDelays.csv")
val df_Unemployment       = load_from_COS("Unemployment.csv")
val df_whiskey            = load_from_COS("whiskey.csv")
// BGG Top 2000 Games.csv については Categories, Mechanics列に null 値が含まれているとグラフ描画に失敗するため null 値を全て"No Data"に置き換え
val df_BGG_Top_2000_Games = load_from_COS("BGG Top 2000 Games.csv").na.fill("No Data", Array("Categories", "Mechanics", "Language"))
val df_cpi                = load_from_COS("cpi.csv")
val df_sierpinski         = load_from_COS("sierpinski.csv")
val df_minard_troops      = load_from_COS("minard-troops.csv")
val df_minard_cities      = load_from_COS("minard-cities.csv")

In [4]:
%%DataFrame --limit 5
df_US_States

Abbr,State,Date,Region,Population,Under 18,Area,Latitude,Longitude,Water,Density,Summer,Winter,Rain,Violent Crimes,Income,Dem/Rep,Presidential Choice,Boys Name,Girls Name
AL,Alabama,1819-12-14,South,4849377,22.8,52420,32.74,-86.84,3.5,92.5,78.6,46.5,58.3,425,41415,-22.2,Romney,William,Eva
AK,Alaska,1959/1/3,Pacific,736732,25.3,665384,63.35,-152.84,16.6,1.1,52.3,2.6,22.5,688,69825,-14.0,Romney,Liam,Emma
AZ,Arizona,1912/2/14,Mountain,6731484,24.1,113990,34.21,-111.6,0.3,59.1,78.1,43.6,13.6,501,46709,-9.1,Romney,Noah,Sophia
AR,Arkansas,1836-06-15,South,2966369,23.8,53179,34.9,-92.44,2.2,55.8,78.8,41.5,50.6,552,38758,-23.7,Romney,Mason,Emma
CA,California,1850-09-09,Pacific,38802500,23.6,163695,37.15,-119.54,5.1,237.0,73.4,46.2,22.2,533,60287,23.1,Obama,Noah,Sophia


In [5]:
%%DataFrame --limit 5
df_AirlineDelays

Result,Late,Month,Day,Count,Fraction
0,0,1,1,154817,0.502386724
1,0,1,1,82024,0.266170825
2,1,1,1,28829,0.09355114
3,1,1,1,19794,0.064232241
4,1,1,1,10728,0.034812745


In [6]:
%%DataFrame --limit 5
df_Unemployment

Series id,Year,Period,Women,Men
LNS14000026,2005,M01,4.7,4.7
LNS14000026,2005,M02,4.7,4.8
LNS14000026,2005,M03,4.6,4.6
LNS14000026,2005,M04,4.6,4.4
LNS14000026,2005,M05,4.7,4.3


In [7]:
// df_whiskey contains null and magic command %%Dataframe can't handle null.
df_whiskey.show(5, false)

+-------------------------------------+------+-------+--------+-----+----+----+---------------+
|Name                                 |Rating|Country|Category|Price|ABV |Age |Brand          |
+-------------------------------------+------+-------+--------+-----+----+----+---------------+
|Canadian Hunter Canadian Whisky      |40    |Canada |Blended |9    |40.0|null|Canadian Hunter|
|Canadian LTD Blended Canadian Whiskey|43    |Canada |Blended |10   |null|null|Canadian LTD   |
|Kellan Irish Whiskey                 |47    |Ireland|Blended |20   |40.0|null|Kellan         |
|Rich & Rare Canadian Whisky          |47    |Canada |Blended |10   |null|null|Rich & Rare    |
|Canadian Mist Blended Canadian Whisky|48    |Canada |Blended |12   |40.0|null|Canadian Mist  |
+-------------------------------------+------+-------+--------+-----+----+----+---------------+
only showing top 5 rows



In [8]:
// df_BGG_Top_2000_Games contains null and magic command %%Dataframe can't handle null.
import org.apache.spark.sql.functions.lit

df_BGG_Top_2000_Games.filter($"Rank".between(lit(1396), lit(1403))).show(false)

+----+------+------------------------------+------+------+---------+-----------+----------+----------+--------+---------------+-----------------------------+----------+---------+----------+------------------+
|Rank|BGG ID|Title                         |Rating|Voters|Published|PlayingTime|MinPlayers|MaxPlayers|Designer|Categories     |Mechanics                    |Expansions|PlayerAge|NumPlayers|Language          |
+----+------+------------------------------+------+------+---------+-----------+----------+----------+--------+---------------+-----------------------------+----------+---------+----------+------------------+
|1396|141932|The Agents                    |6.134 |903   |2013     |40         |2         |5         |null    |Card Game      |Hand Management              |12        |10       |3         |Too much to be fun|
|1397|37208 |Court of the Medici           |6.133 |748   |2009     |25         |2         |2         |null    |Card Game      |Hand Management              |0      

In [9]:
%%DataFrame --limit 5
df_cpi

Series Id,Year,Period,Value
CUUR0000SEEB01,2005,M01,7.9
CUUR0000SEEB01,2005,M02,8.0
CUUR0000SEEB01,2005,M03,7.9
CUUR0000SEEB01,2005,M04,7.9
CUUR0000SEEB01,2005,M05,8.1


In [10]:
%%DataFrame --limit 5
df_sierpinski

a,b
0,15
0,16
15,16
15,12
15,17


In [11]:
%%DataFrame --limit 5
df_minard_troops

long,lat,survivors,direction,group
24.0,54.9,340000,A,1
24.5,55.0,340000,A,1
26.0,54.7,320000,A,1
27.0,54.8,300000,A,1
28.0,54.9,280000,A,1


In [12]:
%%DataFrame --limit 5
df_minard_cities

long,lat,city
24.0,55.0,Kowno
25.3,54.7,Wilna
26.4,54.4,Smorgoni
26.8,54.3,Moiodexno
27.7,55.2,Gloubokoe


## 2. Brunelによるグラフの描画

Scala/Sparkで描画する際には %AddJar でbrunelのライブラリーを読み込んでおく。<br>
ローカルにjarファイルをDownloadして持ってきている場合は https ではなく以下の様に file を使用する。
> %AddJar -magic file:///usr/lpp/IBM/izoda/DSDriver/dv-jdbc-3.1.201706060539/dv-jdbc-3.1.201706060539.jar

In [13]:
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar

Using cached version of spark-kernel-brunel-all-2.3.jar


### 2.1 Basic Charts

**Bar of Counts :** 各リージョンの州の数

In [14]:
%%brunel data('df_US_States') bar x(Region) y(#count)

**Bar of Means (sorted):** 降雨量の降順で並べ替えた各リージョンでの平均降雨量

In [15]:
%%brunel data('df_US_States') bar x(Region) y(Rain) mean(Rain) sort(Rain)

**Clustered Bar Chart :** 各リージョンで大統領を選択した州の数

In [16]:
%%brunel data('df_US_States') bar x(Region, Presidential_Choice) y(#count) color(Presidential_Choice)

**Horizontal Bars :** 平均降雨量を水平バーで表示

In [17]:
%%brunel data('df_US_States') bar x(Region) y(Rain) mean(Rain) sort(Rain) transpose

**Bar of Counts :** 各リージョン毎の収入合計を、選択した大統領別に表示

In [18]:
%%brunel data('df_US_States') bar x(Region) y(Income) sum(Income) color(Presidential_Choice) stack

**Scatterplot :** 50のアメリカの州について夏と冬の気温をプロット

In [19]:
%%brunel data('df_US_States') x(Summer) y(Winter)

**Scatterplot with Color :** 各州の人口と収入を大統領の選択によって色分けをしてプロット

In [20]:
%%brunel data('df_US_States') x(Population) y(Income) color(Presidential_Choice)

**Line Plot :** 月毎の合計の遅延フライト数を線グラフで表示

In [21]:
%%brunel data('df_AirlineDelays') x(Month) y(Late) sum(Late) line

**Series Lines :** 20歳以上の男女別の失業率（2005-2015）

In [22]:
%%brunel data('df_Unemployment') x(#row) y(Women, Men) color(#series) line

**Area Plot :**  月毎の合計フライト数

In [23]:
%%brunel data('df_AirlineDelays') x(Month) y(Count) sum(Count) area

**Pie Chart :** 各リージョンに含まれる州の数の割合

In [24]:
%%brunel data('df_US_States') stack polar bar x("const") y(#count) color(region) legends(none) label(region)

### 2.2 Statistical Graphics

**Linear Fit :** アメリカの州の夏と冬の気温をプロットし、夏の気温から冬の気温を線形回帰で予想した値の直線も一緒に表示

In [25]:
%%brunel data('df_US_States') point x(Summer) y(Winter) + data('df_US_States') line x(Summer) y(Winter) fit(Winter)

**Rectangular Binned Scatterplot :** ウィスキーの評価と熟成年の分布

In [26]:
%%brunel data('df_whiskey') point x(Rating) y(Age) bin(Rating,Age) opacity(#count)

**Smooth Line :**  ウィスキーの評価と値段をプロットし、Adaptive KernelでSmoothingを行った曲線を一緒に表示。

参考： brunelのsmoothオプションの説明
- *smooth:* Smooths the values using an adaptive kernel. If an optional value is provided, it will define the percent of the data to include near each point when smoothing

In [27]:
%%brunel data('df_whiskey') line x(Rating) y(Price) smooth(Price) + data('df_whiskey') point x(Rating) y(Price)

**Median and Inner Quartile Range :** 各リージョン毎の夏の気温の変動を表示　(第一四分位点、第三四分位点を縦棒で表し、中央値をその中の線で表示）

参考： brunelのiqr,medianオプションの説明
- *median* : Median value of the group
- *range* : Distance between min and max values
- *q1, q3* : Lower and Upper quartiles
- *iqr* :Distance between q1 and q3 -- the interquartile range

In [28]:
%%brunel data('df_US_States') bar x(Region) yrange(Summer) iqr(Summer) + data('df_US_States') point x(Region) y(Summer) median(Summer) style('symbol:rect;height:2')

**Histogram :** 夏の気温の度数分布

In [29]:
%%brunel data('df_US_States') bar x(Summer) y(#count) bin(Summer) style("size:100%")

**Brushing/Linking :** カテゴリー毎のウィスキーの値段を棒グラフで表示し、マウスが上に来た場合(mouse over event)にアルコール度数(Alcohol by volume, ABV)と熟成年を表示

補足：コード中のy(Price:linear)はy軸のスケールを対数や冪根をとったスケールに変えず、そのまま線形のスケールを使用するという指定

In [30]:
%%brunel data('df_whiskey') x(Category) y(Price:linear) mean(Price) bar interaction(select:mouseover) color(#selection)
| data('df_whiskey') x(ABV) y(Age) color(#selection) size(#selection) 

### 2.3 Information Visualization

**Heatmap :** 各カテゴリーと機構に対するゲームの数

In [51]:
%%brunel data('df_BGG_Top_2000_Games') x(Categories) y(Mechanics) color(#count:red) style('symbol:rect; size:100%; stroke:none') sort(#count)

**Spark Line :** US の消費者物価指数（2005-2015）

In [61]:
%%brunel data('df_cpi') line x(#row) y(Value) axes(none)

**Bubble Chart :** ゲーム・カテゴリー毎のゲーム・プレイ時間合計

In [54]:
%%brunel data('df_BGG_Top_2000_Games') bubble x(Categories) color(Categories) size(PlayingTime) label(Categories) sum(PlayingTime) legends(none)
:: width=600, height=600

**Tag Cloud :** 平均降雨量によってサイジングされたUSのリージョン

In [34]:
%%brunel data('df_US_States') cloud color(Region) size(Rain) label(Region) mean(Rain) legends(none)

**Treemap :** "使われた言葉”の中に平均のゲーム評価でサイジングし、プレーヤーの平均年齢で色付けしてゲーム・カテゴリーを表示

In [56]:
%%brunel data('df_BGG_Top_2000_Games') treemap x(Language, Categories) color(PlayerAge:continuous) size(Rating) label("Rating: ", Rating) mean(PlayerAge, Rating)
:: width=1000, height=1000

**Chord Chart :** 各国で生産されたウィスキーのカテゴリー

In [59]:
%%brunel data('df_whiskey') chord x(Country) y(Category) color(#count) tooltip(#all)
:: width=600, height=600

**Filtered Bar Chart :** 熟成年とアルコール度数でフィルターされたカテゴリー毎のウィスキーの平均価格

In [60]:
%%brunel data('df_whiskey') x(Category) y(Price:linear) mean(Price) bar filter(Age, ABV)

### 2.4 Maps and Networks

**Map of the USA :** 州で分け、州名をラベルとしてつけたアメリカの地図を表示

In [68]:
%%brunel data('df_US_States') map key(state) label(state)
:: width=600, height=600

**Rainfall over the United States :** 年間の平均降雨量を表示しているアメリカのコロプレス地図

In [69]:
%%brunel data('df_US_States') map color(rain:[reds, blues]) key(state) label(state)
:: width=600, height=600

**Multiple Attributes of States :** 州の枠は人口を濃淡で表示し、その中の丸のサイズで冬の平均気温を、丸の色で州立記念日を表示

In [70]:
%%brunel data('df_US_States') map color(population:greens) key(state) + point map color(date:[black]) size(winter:400%) key(state)

**Minard's Map of Napoleon's march on Russia :** パスの太さは軍隊の規模に比例。都市名は現在の物を表示

In [73]:
%%brunel data('df_minard_troops') map('russia', 'belarus', 'lithuania')
+ path x(long) y(lat) color(direction) size(survivors:400%) split(group) legends(none)
+ map(labels)

**Europe, highlighting France :** 特にデータなしで背景の地図を表示

In [81]:
%%brunel map('Europe', 'France')

**The Sierpinski Gasket :** シェルピンスキ・データのレイアウト -- 純粋に抽象的なネットワーク

Cookbookでは以下のようになっているがこれは実行出来なかったため、networkの指定を修正。
> %%brunel data('df_sierpinski') edge key(a, b) + network y(a, b) color(#values) label(#values) legends(none)

networkにはノードのキーをもつ列を含んだデータを、edgeにはどのノードとどのノードが接続されるかを示す2つの列を含んだデータを渡すが、Cookbookの例ではnetworkにy(a, b)と2つの列を渡している。そこで、a列とb列に含まれているノード番号を1つにマージしたデータフレームを作成し、これをnetworkで指定するようにする。

In [90]:
val df_sierpinski_nodes = df_sierpinski.select($"a" as "ID").union(df_sierpinski.select($"b" as "ID")).distinct().orderBy($"ID")

In [97]:
%%brunel data('df_sierpinski') edge key(a, b) + data('df_sierpinski_nodes') network key(ID) color(ID) label(ID) legends(none)
:: width=700, height=700